<a href="http://landlab.github.io"><img style="float: left; height: 175px; width: 175px" src="../landlab_logo.jpg"></a> <h3 style="margin: 117px 0 0 185px; font-weight: 300;">a toolkit for modeling earth surface processes</h3>

# Landlab grids

## Understanding the topology of Landlab grids

All grids consist of two interlocked sets of *points* joined by *lines* outlining *areas*. The data points are called **nodes**, which are joined by **links** which outline **patches**. Each node within the interior of the grid lies at the geometric center of the area of a **cell**. The cell's edges are **faces**, and the point at the edge of each face is a **corner**.

Note that this kind of scheme requires one set of features to be "dominant" over the other; i.e., either not every node has a cell, *or* not every link is crossed by a face. Both cannot be true, because one or other set of features has to define the edge of the grid. Landlab assumes that the **node set is primary**, so there are always more nodes than corners; more links than faces; and more patches than cells.

Each of these sets of *"elements"* has its own set of IDs. These IDs are what allow us to index the various Landlab fields which store spatial data. Each feature is ordered by **x, then y**. The origin is always at the bottom left node, unless you choose to move it (`grid.move_origin`)... except in the specific case of a radial grid, where logic and symmetry dictates it must be the central node.

The final thing to know is that **lines have direction**. This lets us record fluxes on the grid by associating them with, and mapping them onto, the links (or, much less commonly, the faces). All lines point into the **upper right quadrant**. So, on our raster, this means the horizontal links point east an the vertical links point north.

So, for reference, our raster grid looks like this:


    NODES:                       LINKS:                       PATCHES:
    8 ----- 9 ---- 10 ---- 11    *--14-->* -15-->* -16-->*    * ----- * ----- * ----- *
    |       |       |       |    ^       ^       ^       ^    |       |       |       |
    |       |       |       |   10      11      12      13    |   3   |   4   |   5   |
    |       |       |       |    |       |       |       |    |       |       |       |
    4 ----- 5 ----- 6 ----- 7    * --7-- * --8-- * --9-- *    * ----- * ----- * ----- *
    |       |       |       |    ^       ^       ^       ^    |       |       |       |
    |       |       |       |    3       4       5       6    |   0   |   1   |   2   |
    |       |       |       |    |       |       |       |    |       |       |       |
    0 ----- 1 ----- 2 ----- 3    * --0-->* --1-->* --2-->*    * ----- * ----- * ----- *

    CELLS:                       FACES:                       CORNERS:
    * ----- * ----- * ----- *    * ----- * ----- * ----- *    * ----- * ----- * ----- *
    |       |       |       |    |       |       |       |    |       |       |       |
    |   . ----- . ----- .   |    |   . --5-->. --6-->.   |    |   3 ----- 4 ----- 5   |
    |   |       |       |   |    |   ^       ^       ^   |    |   |       |       |   |
    * --|   0   |   1   |-- *    * --2       3       4-- *    * --|       |       |-- *
    |   |       |       |   |    |   |       |       |   |    |   |       |       |   |
    |   . ----- . ----- .   |    |   . --0-->. --1-->.   |    |   0 ----- 1 ----- 2   |
    |       |       |       |    |       |       |       |    |       |       |       |
    * ----- * ----- * ----- *    * ----- * ----- * ----- *    * ----- * ----- * ----- *



Landlab supports a range of grid types. These include both rasters (with both square and rectangular cells), and a range of structured and unstructured grids based around the interlocking polygons and triangles of a Voronoi-Delaunay tesselation (radial, hexagonal, and irregular grids).

Below, we'll start looking at some of the features of a **rectangular raster**.

### First, we'll create a grid

In [ ]:
%matplotlib inline

# Import relevant pieces of landlab
from landlab import RasterModelGrid
from landlab.plot import imshow_grid

# Import some important other python packages
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Create a rectangular raster with 4 rows, 5 columns, and 10 unit spacing
rmg = RasterModelGrid((4,5), 10.0)

In [ ]:
# How many nodes are there?
rmg.number_of_nodes

In [ ]:
# How many links?
rmg.number_of_links

In [ ]:
# How many rows?
rmg.number_of_node_rows

In [ ]:
# How many columns?
rmg.number_of_node_columns

In [ ]:
# How many core nodes?
rmg.number_of_core_nodes

In [ ]:
# How many cells?
rmg.number_of_cells

### Let's attach elevation data to the grid!

In [ ]:
# Create a random array of elevations
z = 10*np.random.rand(rmg.number_of_nodes)

# Attach the elevation array to the grid as a new field called topographic__elevation
rmg.add_field('node', 'topographic__elevation', z)

We can plot the elevation field to look at our data graphically.

In [ ]:
imshow_grid(rmg, 'topographic__elevation', plot_name='Example grid', var_name='Elevation (m)')
plt.show()

### Explore node/link structure and new topographic__elevation field

In [ ]:
# Quick status definitions:
#     CORE_NODE = 0
#     FIXED_VALUE_BOUNDARY = 1
#     FIXED_GRADIENT_BOUNDARY = 2
#     TRACK_CELL_BOUNDARY = 3
#     CLOSED_BOUNDARY = 4
# Most often you will see 0, 1, and 4

rmg.status_at_node[13]

In [ ]:
rmg.links_at_node[13]

In [ ]:
rmg.link_dirs_at_node[13]

In [ ]:
rmg.node_at_link_tail[20]

In [ ]:
rmg.at_node['topographic__elevation'][13]

We can change the elevation at node 13 using the following:

In [ ]:
rmg.at_node['topographic__elevation'][13] = 100

# We can also use z[13] = 100 to do this!

In [ ]:
# Let's check that we actually changed it
rmg.at_node['topographic__elevation'][13]

# Landlab components

Now, we can walk through the stages of creating and running a Landlab model using the Landlab component library.

We are going to develop two models:  
1. A single component driver implementing just linear diffusion  
2. A three-component driver implementing linear diffusion, flow routing, and stream power incision.

## The basics: using one component

Let's begin with a very basic one-component diffusion model.

Quick note: All Landlab component names are formatted in CamelCaseLikeThis.

In [ ]:
# Import the LinearDiffuser component
from landlab.components import LinearDiffuser

In [ ]:
# Create a square model grid
mg = RasterModelGrid((80, 80), 5.)

# Add our elevation field. 
# Note that this is slightly different than what we did in the previous example!
z = mg.add_zeros('node', 'topographic__elevation')

How did we know that elevation was a necessary input field for the LinearDiffuser component? Well, firstly because we read the component documentation (**always do this!**), but secondly we can get a reminder using the Landlab Component Standard Interface:

In [ ]:
LinearDiffuser.input_var_names

Note we didn't have to instantiate the component to be able to do this! Other standard properties are `output_var_names` and `optional_var_names`; pass an input or output name to `var_loc`, `var_type`, `var_units`, and `var_definition` to get the centering ('node', 'link', etc), array dtype (float, int), units (meters, etc) and a descriptive string, respectively. `var_help` will give you a lot of this information at once:

In [ ]:
LinearDiffuser.var_help('topographic__elevation')

It's also a good idea to set the grid boundary conditions before component instantiation. Let's have fixed value top and bottom and closed left and right:

In [ ]:
# Set boundary conditions! These built-in methods for Landlab sure are nifty, huh? 
mg.set_closed_boundaries_at_grid_edges(True, False, True, False)
mg.set_fixed_value_boundaries_at_grid_edges(False, True, False, True)

Now we can move on to using the component!  
All components within landlab share a similar interface. We'll examine how it looks first on the diffusion component.

You need two pieces main pieces to run a landlab component: 
1. The component instantiation
2. The component run_one_step() method 

### Component instantiation
The standard signature to instantiate a component looks like this:

    MyComponent(grid, input1=default1, input2=default2, input3=default3, ..., **kwds)

Because defaults are provided, you can instantiate a component with default values very simply; the diffuser requires only that a `linear_diffusity` be supplied:

In [ ]:
lin_diffuse = LinearDiffuser(mg, linear_diffusivity=0.2)

### Running the component
Now we're ready to run the component! All Landlab components have a standard run method named `run_one_step`, and it looks like this:

    my_comp.run_one_step([dt], other_inputA=defaultA, ...)

If the component is time-dependent, `dt`, the timestep, will be the first argument. Subsequent keywords will typically be flags that control the way the component runs, and typically can be left as their default values. Note that nothing is returned from a run method like this, but *the grid fields are updated*.

This `dt` is properly thought of as the *external model timestep*; it controls the frequency at which the various Landlab components you're implementing can exchange information with each other and with the driver (e.g., frequency at which uplift steps are added to the grid). If your model has a stability condition which demands a shorter timestep, the external timestep will be subdivided internally down to this shorter timescale.

So let's do it. It's up to you as the model designer to make sure your driver script accounts properly for the total time the model runs. Here, we want to run for 200000 years with a timestep of 1000 years, with an uplift rate of 0.001 m/y. So:

In [ ]:
total_t = 200000.
dt = 1000.
uplift_rate = 0.001
nt = int(total_t // dt)
# Note: if we didn't know a priori that there are a round number of steps dt in the
# total time, we'd have to take care to account for the "extra" time.

# Run the model
for i in range(nt):
    lin_diffuse.run_one_step(dt)
    
    z[mg.core_nodes] += uplift_rate * dt  # add the uplift
    
    # Print some output to let us know the model *is* running
    if i % 50 == 0:
        print(i*dt)

Note that we're using `z` to input the uplift here, which we bound to the Landlab field `mg.at_node['topographic__elevation]` when we instantiated that field.

Now plot the output!  

In [ ]:
# Create a figure and plot the elevations
imshow_grid(mg, 'topographic__elevation', grid_units = ['m','m'],
            var_name='Elevation (m)')
plt.show()

In [ ]:
# Let's look at a cross-section of this!

elev_rast = mg.node_vector_to_raster(z) #Convert elevation to a raster
ycoord_rast = mg.node_vector_to_raster(mg.node_y) #Convert the y coords to a raster
ncols = mg.number_of_node_columns #How many columns?

plt.plot(ycoord_rast[:, int(ncols // 2)], elev_rast[:, int(ncols // 2)])
plt.xlabel('Horizontal distance (m)')
plt.ylabel('Vertical distance (m)')
plt.title('topographic__elevation cross section')
plt.show()

# Running two or more components

Now we're going to take a similar approach but this time combine the outputs of three distinct Landlab components: the linear diffuser, the flow router, and the stream power eroder. For clarity, we're going to repeat the whole process from the start.

In [ ]:
# Import components we don't already have
from landlab.components import FlowAccumulator, FastscapeEroder

In [ ]:
# Initialize some parameters
nrows = 100
ncols = 100
dx = 0.02 #km
uplift_rate = 0.001 #km/yr
total_t = 100. #yr
dt = 0.5 #yr

nt = int(total_t // dt) #this is how many loops we'll need
uplift_per_step = uplift_rate * dt

Now instantiate the grid, set the initial conditions, and set the boundary conditions:

In [ ]:
# Create our grid and add topographic__elevation field
mg = RasterModelGrid((nrows, ncols), dx)
z = mg.add_zeros('node', 'topographic__elevation')

# Add some roughness, as this lets "natural" channel planforms arise
initial_roughness = np.random.rand(z.size)/100000.
z += initial_roughness

# Set boundary conditions
mg.set_closed_boundaries_at_grid_edges(True, False, True, False)
mg.set_fixed_value_boundaries_at_grid_edges(False, True, False, True)

So far, so familiar.

Now we're going to instantiate all our components.

In [ ]:
# Instantiate the flow routing component using the defaults
fa = FlowAccumulator(mg)

# Instantiate the erosion component, changing some of the defaults
fsc = FastscapeEroder(mg, K_sp=0.3, m_sp=0.5, n_sp=1)

# Instantiate the linear diffusion component, changing some of the defaults
lin_diffuse = LinearDiffuser(mg, rock_density=2.7, sed_density=2.7,linear_diffusivity=0.0001)

And now we run! We're going to run once with the diffusion and once without.

In [ ]:
# Run model without diffusion first

for i in range(nt):
    # lin_diffuse.run_one_step(dt) no diffusion this time
    fa.run_one_step() # run_one_step for fa isn't time sensitive, so it doesn't take dt as input
    fsc.run_one_step(dt)
    
    mg.at_node['topographic__elevation'][mg.core_nodes] += uplift_per_step # add the uplift
    
    if i % 20 == 0:
        print ('Completed loop %d' % i)

In [ ]:
# Look at our beautiful plot!
imshow_grid(mg, 'topographic__elevation', plot_name='No linear diffusion',
            grid_units=['km','km'], var_name='Elevation (km)')
plt.show()

And now let's reset the grid elevations and do everything again, but this time, with the diffusion turned *on*:

In [ ]:
# Reset to the initial elevation
z[:] = initial_roughness

# Run model with linear diffusion!
for i in range(nt):
    lin_diffuse.run_one_step(dt)  #with diffusion this time
    fa.run_one_step()
    fsc.run_one_step(dt)
    
    mg.at_node['topographic__elevation'][mg.core_nodes] += uplift_per_step #add the uplift
    
    if i % 20 == 0:
        print ('Completed loop %d' % i) #print something so we know the model is running!

In [ ]:
# Another pretty plot!
imshow_grid(mg, 'topographic__elevation', plot_name='With linear diffusion',
            grid_units=['km','km'], var_name='Elevation (km)')
plt.show()

Ta-da! You can now create model drivers to use Landlab components!

### Click here for more <a href="https://github.com/landlab/landlab/wiki/Tutorials">Landlab tutorials</a>